# Lab 8
#### Nanyang Technology University, SC1015 Introduction to Data Science and Artificial Intelligence
#### Tutorial: FCSA, Team: 1, Teaching Assistant: Mr. GIRISH KUMAR DEEPANK.
#### Authors:
- Gay Ming Kai (C230114@e.ntu.edu.sg)
- Jacob Ong Jia Chun (JONG163@e.ntu.edu.sg)
- Lim Zu Liang (E230184@e.ntu.edu.sg)

Note: Please feel free to reach out to us should there be any queries or concerns!

## Chapter 1: Crossy Roads (Failed Attempt)

### 1.1 User Playable

In [30]:
import random
import time
import sys
import keyboard
from IPython.display import clear_output

CLEAR_SCREEN = "\033[2J\033[H"
MOVE_UP = "\033[1A"
MOVE_DOWN = "\033[1B"
MOVE_RIGHT = "\033[1C"
MOVE_LEFT = "\033[1D"

def display_game(player_position, obstacle_positions, score):
    game_display = [['🌳'] + (["  "] * 20) + ['🌳'] for _ in range(10)]
    game_display[player_position[0]][player_position[1]] = '🚘'
    for pos, size in obstacle_positions:
        for i in range(size[0]):
            for j in range(size[1]):
                if pos[0] + i >= 0 and pos[0] + i < 10 and pos[1] + j >= 0 and pos[1] + j < 20:
                    game_display[pos[0] + i][pos[1] + j] = '🌳'
    sys.stdout.write(CLEAR_SCREEN)
    for row in game_display:
        for char in row:
            sys.stdout.write(char)
        sys.stdout.write("\n")
    sys.stdout.write(f"Score: {score}\n")
    sys.stdout.flush()

def move_player_left(player_position):
    if player_position[1] > 1:
        player_position[1] -= 1

def move_player_right(player_position):
    if player_position[1] < 20:
        player_position[1] += 1

def generate_obstacle():
    size = random.randint(1, 1)
    pos = [0, random.randint(0, 20 - size)]
    return (pos, (size, size))

def move_obstacle(obstacle_positions):
    for idx, (pos, size) in enumerate(obstacle_positions):
        pos[0] += 1
        if pos[0] >= 10:
            del obstacle_positions[idx]

def check_collision(player_position, obstacle_positions):
    for pos, size in obstacle_positions:
        for i in range(size[0]):
            for j in range(size[1]):
                if pos[0] + i == player_position[0] and pos[1] + j == player_position[1]:
                    return True
    return False

def endless_runner():
    player_position = [9, 10]
    obstacle_positions = []
    game_speed = 0.1
    obstacle_spawn_rate = 0.2
    score = 0
    last_score_update_time = time.time()
    while True:
        if random.random() < obstacle_spawn_rate:
            obstacle_positions.append(generate_obstacle())
        move_obstacle(obstacle_positions)
        if check_collision(player_position, obstacle_positions):
            sys.stdout.write(f"Game Over! You collided with an obstacle. Final Score: {score}\n")
            sys.stdout.flush()
            return
        current_time = time.time()
        if current_time - last_score_update_time >= 1:
            score += 1
            last_score_update_time = current_time
        display_game(player_position, obstacle_positions, score)
        time.sleep(game_speed)
        if keyboard.is_pressed('left'):
            move_player_left(player_position)
        elif keyboard.is_pressed('right'):
            move_player_right(player_position)
        clear_output(wait=True)

if __name__ == "__main__":
    try:
        endless_runner()
    except KeyboardInterrupt:
        sys.exit(0)


Game Over! You collided with an obstacle. Final Score: 7


### 1.2 Automated Playing

In [29]:
from IPython.display import clear_output
import random
import time
import sys

CLEAR_SCREEN = "\033[2J\033[H"
MOVE_UP = "\033[1A"
MOVE_DOWN = "\033[1B"
MOVE_RIGHT = "\033[1C"
MOVE_LEFT = "\033[1D"

def display_game(player_position, obstacle_positions, score):
    game_display = [['🌳'] + (["  "] * 20) + ['🌳'] for _ in range(10)]

    game_display[player_position[0]][player_position[1]] = '🚘'

    for pos, size in obstacle_positions:
        for i in range(size[0]):
            for j in range(size[1]):
                if pos[0] + i >= 0 and pos[0] + i < 10 and pos[1] + j >= 0 and pos[1] + j < 20:
                    game_display[pos[0] + i][pos[1] + j] = '🌳'

    sys.stdout.write(CLEAR_SCREEN)
    for row in game_display:
        for char in row:
            sys.stdout.write(char)
        sys.stdout.write("\n")
    sys.stdout.write(f"Score: {score}\n")
    sys.stdout.flush()

def move_player_left(player_position):
    if player_position[1] > 1:
        player_position[1] -= 1

def move_player_right(player_position):
    if player_position[1] < 20:
        player_position[1] += 1

def generate_obstacle(obstacle_positions, last_positions, max_gap=2):
    pos = [0, random.randint(0, 20 - 1)]
    while any(abs(pos[1] - last_pos[1]) < max_gap for last_pos in last_positions):
        pos = [0, random.randint(0, 20)]

    obstacle_positions.append((pos, (1, 1)))

    last_positions.append(pos)
    if len(last_positions) > max_gap:
        last_positions.pop(0)

def move_obstacle(obstacle_positions):
    for idx, (pos, size) in enumerate(obstacle_positions):
        pos[0] += 1
        if pos[0] >= 10:
            del obstacle_positions[idx]

def check_collision(player_position, obstacle_positions):
    for pos, size in obstacle_positions:
        for i in range(size[0]):
            for j in range(size[1]):
                if pos[0] + i == player_position[0] and pos[1] + j == player_position[1]:
                    return True
    return False

def autoplay(player_position, obstacle_positions):
    obstacles_above = [pos for pos, _ in obstacle_positions if pos[0] == player_position[0] - 1 and pos[1] == player_position[1]]
    
    if obstacles_above:
        move_direction = random.choice(['left', 'right'])
        if move_direction == 'left' and player_position[1] > 1 and not check_collision([player_position[0], player_position[1] - 1], obstacle_positions):
            player_position[1] -= 1
        elif move_direction == 'right' and player_position[1] < 20 and not check_collision([player_position[0], player_position[1] + 1], obstacle_positions):
            player_position[1] += 1

def endless_runner():
    player_position = [9, 10]
    obstacle_positions = []
    last_positions = []
    game_speed = 0.02
    obstacle_spawn_rate = 0.6
    score = 0
    last_score_update_time = time.time()

    while True:
        if random.random() < obstacle_spawn_rate:
            generate_obstacle(obstacle_positions, last_positions)
        
        move_obstacle(obstacle_positions)
        
        if check_collision(player_position, obstacle_positions):
            sys.stdout.write(f"Game Over! You collided with an obstacle. Final Score: {score}\n")
            sys.stdout.flush()
            return
        
        current_time = time.time()
        if current_time - last_score_update_time >= 1:
            score += 1
            last_score_update_time = current_time
        
        autoplay(player_position, obstacle_positions)
        display_game(player_position, obstacle_positions, score)
        time.sleep(game_speed)
        clear_output(wait=True)

if __name__ == "__main__":
    try:
        endless_runner()
    except KeyboardInterrupt:
        sys.exit(0)


🌳                                        🌳
🌳            🌳                          🌳
🌳        🌳                              🌳
🌳                                        🌳
🌳                                        🌳
🌳                    🌳                  🌳
🌳                                        🌳
🌳      🌳                                🌳
🌳                        🌳              🌳
🌳                🚘                      🌳
Score: 3


AttributeError: 'tuple' object has no attribute 'tb_frame'

---
The automated playing algorithm is a rule-based approach where actions will be undertaken based on the rules set by us that are based on game states such as distance to obstacles, speed, and size. Crossy Road is an "Endless Runner", which is typically trained using reinforcement learning specifically Q-Learning, or done using genetic algorithms which we could not use.

The reason traditional uninformed search algorithms are avoided is because there is no specific goal state to reach as the goal of the program is to survive as long as possible, unlike uninformed search algorithms like BFS or DFS which are used to find the shortest path to the goal state. Moreover, in "Endless Runner" games, the environment is real-time and is procedurally generated as the game progresses, which makes it difficult to build a complete graph or tree of the game environment which is a necessary step for uninformed search algorithms. 

Informed search algorithms like greedy best-first search and A* search algorithm are also not suitable for training our agent as they both require evaluating heuristic functions. Since the environment is dynamic, the complexity and computational power required to keep on updating the heuristic is very prohibitive. Furthermore, with the random events and limitation in visibility, it is very difficult to keep on generating heuristic functions to guide it as after a frame, the previous heuristic function becomes irrelevant due to new obstacles being generated and your current position being updated. Lastly, both greedy best-first search algorithm and A* search prioritize paths based on their estimated cost to the goal as such this can lead to both algorithms not moving from their original position much which will miss out on effective strategies that are not apparent based on heuristic estimates. 

The first algorithm is based on a rule-based system together with a small incorporation of greedy best-first search. By incorporating greedy best-first search algorithm into our algorithms where the environment is limited to the vicinity of our agent so that we can identify if we can dodge left or right depending if there is an obstacle in front. For every node, there would be 3 child nodes/states that it can undertake, being not moving, moving left, and moving right. But the "graph" will only consist of depth 2 where the 3 child nodes/states will end in the goal. The weights will be distributed as staying still being the highest weight followed by moving left or right unless there is an obstacle coming up where the weight of staying still will be lowered to allow the best path to be either move left or right.

Since we could not implement any proper search algorithm, this endeavor ended up a failure but we definitely learned a lot.

---

## Chapter 2: The Wolf, Goat, and Cabbage (Success)

This notebook covers the basic search algorithms for our new problem, **Wolf-Goat-Cabbage**.

The search algorithms we will be showing first are 
- `breadth_first_search`
- `depth_first_search`
- `uniformed_cost_search`
- `depth_limited_search`
- `iterative_deepening`
- `greedy_best_first_search`
- `a_star_search`

"Wolf-Goat-Cabbage" is a classic logic puzzle that involves transporting a wolf, a goat, and a cabbage across a river, but with certain constraints to prevent conflicts.

#### Actors
- Farmer & Boat for transporting
- Wolf
- Goat
- Cabbage

#### Environment
- The left river bank (Starting Zone)
- The River
- The right river bank (Goal Zone)

#### Objective
Your goal is to figure out a sequence of crossings that gets all 3 items safely to the other side of the river without any of them being eaten.


#### Rules
- The Wolf cannot be left alone with Goat
- The Goat cannot be left alone with Cabbage


### Breadth-First Search

In [7]:
class WGC_Node:
    incompatibilities = [
        ["c", "g", "w"],
        ["g", "w"],
        ["c", "g"]
    ]

    def __init__(self, west=["w", "c", "g"], east=[], boat_side=False, children=[]):
        self.west = west
        self.east = east
        self.boat_side = boat_side
        self.children = children

    def __str__(self):
        return str(self.west) + str(self.east) + ("Left" if not self.boat_side else "Right")

    def generate_children(self, previous_states, parent_map):
        children = []
        if not self.boat_side:
            for i in self.west:
                new_west = self.west[:]
                new_west.remove(i)
                new_east = self.east[:]
                new_east.append(i)
                if sorted(new_west) not in WGC_Node.incompatibilities and not WGC_Node.state_in_previous(previous_states, new_west, new_east, not self.boat_side):
                    child = WGC_Node(new_west, new_east, not self.boat_side, [])
                    children.append(child)
                    parent_map[child] = self
            if sorted(self.west) not in WGC_Node.incompatibilities and not WGC_Node.state_in_previous(previous_states, self.west[:], self.east[:], not self.boat_side):
                child = WGC_Node(self.west[:], self.east[:], not self.boat_side, [])
                children.append(child)
                parent_map[child] = self
        else:
            for i in self.east:
                new_west = self.west[:]
                new_west.append(i)
                new_east = self.east[:]
                new_east.remove(i)
                if sorted(new_east) not in WGC_Node.incompatibilities and not WGC_Node.state_in_previous(previous_states, new_west, new_east, not self.boat_side):
                    child = WGC_Node(new_west, new_east, not self.boat_side, [])
                    children.append(child)
                    parent_map[child] = self
            if sorted(self.east) not in WGC_Node.incompatibilities and not WGC_Node.state_in_previous(previous_states, self.west[:], self.east[:], not self.boat_side):
                child = WGC_Node(self.west[:], self.east[:], not self.boat_side, [])
                children.append(child)
                parent_map[child] = self
        self.children = children

    @staticmethod
    def state_in_previous(previous_states, west, east, boat_side):
        return any(
            sorted(west) == sorted(i.west) and
            sorted(east) == sorted(i.east) and
            boat_side == i.boat_side
            for i in previous_states
        )

def BFS(root_node):
    to_visit = [root_node]
    node = root_node
    previous_states = []
    parent_map = {root_node: None}
    while to_visit:
        node = to_visit.pop()
        if not WGC_Node.state_in_previous(previous_states, node.west, node.east, node.boat_side):
            previous_states.append(node)
        node.generate_children(previous_states, parent_map)
        to_visit = to_visit + node.children
        if sorted(node.east) == ["c", "g", "w"]:
            solution = []
            while node is not None:
                solution = [node] + solution
                node = parent_map[node]
            return solution
    return None

if __name__ == "__main__":
    root = WGC_Node()
    solution = BFS(root)
    print("BFS solution:")
    for i, node in enumerate(solution):
        left_side = f"[{', '.join(node.west)}]" if node.west else "[]"
        right_side = f"[{', '.join(node.east)}]" if node.east else "[]"
        direction = 'Left' if not node.boat_side else 'Right'
        print(f"Step {i + 1}: {left_side:<10} {right_side:<10} {direction:<10}")

BFS solution:
Step 1: [w, c, g]  []         Left      
Step 2: [w, c]     [g]        Right     
Step 3: [w, c]     [g]        Left      
Step 4: [w]        [g, c]     Right     
Step 5: [w, g]     [c]        Left      
Step 6: [g]        [c, w]     Right     
Step 7: [g]        [c, w]     Left      
Step 8: []         [c, w, g]  Right     


### Depth First Search

In [8]:
def DFS(root_node):
    to_visit = [root_node]
    node = root_node
    previous_states = []
    parent_map = {root_node: None}
    while to_visit:
        node = to_visit.pop()
        if not WGC_Node.state_in_previous(previous_states, node.west, node.east, node.boat_side):
            previous_states.append(node)
        node.generate_children(previous_states, parent_map)
        to_visit = node.children + to_visit

        if sorted(node.east) == ["c", "g", "w"]:
            solution = []
            while node is not None:
                solution = [node] + solution
                node = parent_map[node]
            return solution
    return None

if __name__ == "__main__":
    root = WGC_Node()
    solution = DFS(root)
    print("DFS solution:")
    for i, node in enumerate(solution):
        left_side = f"[{', '.join(node.west)}]" if node.west else "[]"
        right_side = f"[{', '.join(node.east)}]" if node.east else "[]"
        direction = 'Left' if not node.boat_side else 'Right'
        print(f"Step {i + 1}: {left_side:<10} {right_side:<10} {direction:<10}")

DFS solution:
Step 1: [w, c, g]  []         Left      
Step 2: [w, c]     [g]        Right     
Step 3: [w, c]     [g]        Left      
Step 4: [w]        [g, c]     Right     
Step 5: [w, g]     [c]        Left      
Step 6: [g]        [c, w]     Right     
Step 7: [g]        [c, w]     Left      
Step 8: []         [c, w, g]  Right     


### Uniform Cost Search

In [28]:
import heapq

def uniform_cost_search(root_node):
    heap = [(0, id(root_node), root_node)]  # Priority queue with cost and node id
    visited = set()
    parent_map = {id(root_node): None}
    while heap:
        cost, _, node = heapq.heappop(heap)
        if id(node) in visited:
            continue
        visited.add(id(node))
        if sorted(node.east) == ["c", "g", "w"]:
            solution = []
            while node is not None:
                solution = [node] + solution
                parent_id = parent_map.get(id(node))
                node = parent_map.get(parent_id) if parent_id is not None else None
            return solution
        node.generate_children([], parent_map)
        for child in node.children:
            child_id = id(child)
            if child_id not in visited:
                heapq.heappush(heap, (cost + 1, child_id, child))
                parent_map[child_id] = id(node)
    return None

# Usage
root = WGC_Node()
solution = uniform_cost_search(root)
print("Uniform-Cost Search solution:")
print("[",end='')
for node in solution:
    print(node, '\b, ', end='')
print("\b\b]")


Uniform-Cost Search solution:
[1963185127568, []['w', 'c', 'g']Right]


### Depth Limited Search

In [10]:
def depth_limited_search(node, depth_limit, parent_map):
    if depth_limit == 0:
        return None
    if sorted(node.east) == ["c", "g", "w"]:
        solution = []
        while node is not None:
            solution = [node] + solution
            node = parent_map[node]
        return solution
    node.generate_children([], parent_map)
    for child in node.children:
        result = depth_limited_search(child, depth_limit - 1, parent_map)
        if result is not None:
            return result
    return None

# Usage
depth_limit = 10  # Set desired depth limit
parent_map = {root: None}
dls_solution = depth_limited_search(root, depth_limit, parent_map)
print("Depth-Limited Search solution with depth limit {}:".format(depth_limit))
for i,node in enumerate(dls_solution):
    print(f"Step {i + 1}: {node}")



Depth-Limited Search solution with depth limit 10:
Step 1: ['w', 'c', 'g'][]Left
Step 2: ['w', 'c']['g']Right
Step 3: ['w', 'c', 'g'][]Left
Step 4: ['w', 'c']['g']Right
Step 5: ['w', 'c']['g']Left
Step 6: ['c']['g', 'w']Right
Step 7: ['c', 'g']['w']Left
Step 8: ['g']['w', 'c']Right
Step 9: ['g']['w', 'c']Left
Step 10: []['w', 'c', 'g']Right


### Iterative Deepening Search

In [11]:
def iterative_deepening(root_node, max_depth):
    for depth_limit in range(1, max_depth + 1):
        parent_map = {root_node: None}
        solution = depth_limited_search(root_node, depth_limit, parent_map)
        if solution is not None:
            return solution
    return None

# Usage
max_depth = 20  # Set desired maximum depth
iterative_solution = iterative_deepening(root, max_depth)
print("Iterative Deepening solution with max depth {}: ".format(max_depth))

    
for i, node in enumerate(iterative_solution):
        print(f"Step {i + 1}: {node}")


Iterative Deepening solution with max depth 20: 
Step 1: ['w', 'c', 'g'][]Left
Step 2: ['w', 'c']['g']Right
Step 3: ['w', 'c']['g']Left
Step 4: ['c']['g', 'w']Right
Step 5: ['c', 'g']['w']Left
Step 6: ['g']['w', 'c']Right
Step 7: ['g']['w', 'c']Left
Step 8: []['w', 'c', 'g']Right


### Greedy Best First Search

In [12]:
import heapq

class WGC_Node:
    incompatibilities = [
        ["c", "g", "w"],
        ["g", "w"],
        ["c", "g"]
    ]

    def __init__(self, west=["w", "c", "g"], east=[], boat_side=False, children=[]):
        self.west = west
        self.east = east
        self.boat_side = boat_side
        self.children = children

    def __str__(self):
        return str(self.west) + str(self.east) + ("Left" if not self.boat_side else "Right")

    def __lt__(self, other):
        return self.heuristic() < other.heuristic()

    def generate_children(self, previous_states, parent_map):
        children = []
        if not self.boat_side:
            for i in self.west:
                new_west = self.west[:]
                new_west.remove(i)
                new_east = self.east[:]
                new_east.append(i)
                if sorted(new_west) not in WGC_Node.incompatibilities and not WGC_Node.state_in_previous(previous_states, new_west, new_east, not self.boat_side):
                    child = WGC_Node(new_west, new_east, not self.boat_side, [])
                    heapq.heappush(children, child)
                    parent_map[child] = self
            if sorted(self.west) not in WGC_Node.incompatibilities and not WGC_Node.state_in_previous(previous_states, self.west[:], self.east[:], not self.boat_side):
                child = WGC_Node(self.west[:], self.east[:], not self.boat_side, [])
                heapq.heappush(children, child)
                parent_map[child] = self
        else:
            for i in self.east:
                new_west = self.west[:]
                new_west.append(i)
                new_east = self.east[:]
                new_east.remove(i)
                if sorted(new_east) not in WGC_Node.incompatibilities and not WGC_Node.state_in_previous(previous_states, new_west, new_east, not self.boat_side):
                    child = WGC_Node(new_west, new_east, not self.boat_side, [])
                    heapq.heappush(children, child)
                    parent_map[child] = self
            if sorted(self.east) not in WGC_Node.incompatibilities and not WGC_Node.state_in_previous(previous_states, self.west[:], self.east[:], not self.boat_side):
                child = WGC_Node(self.west[:], self.east[:], not self.boat_side, [])
                heapq.heappush(children, child)
                parent_map[child] = self
        self.children = children

    @staticmethod
    def state_in_previous(previous_states, west, east, boat_side):
        return any(
            sorted(west) == sorted(i.west) and
            sorted(east) == sorted(i.east) and
            boat_side == i.boat_side
            for i in previous_states
        )

    def heuristic(self):
        return len(self.west) + len(self.east)


def gbfs(root_node):
    to_visit = [root_node]
    node = root_node
    previous_states = []
    parent_map = {root_node: None}
    while to_visit:
        node = heapq.heappop(to_visit)
        if not WGC_Node.state_in_previous(previous_states, node.west, node.east, node.boat_side):
            previous_states.append(node)
        node.generate_children(previous_states, parent_map)
        to_visit = node.children + to_visit
        if sorted(node.east) == ["c", "g", "w"]:
            solution = []
            while node is not None:
                solution = [node] + solution
                node = parent_map[node]
            return solution
    return None


if __name__ == "__main__":
    root = WGC_Node()
    gbfs_solution = gbfs(root)
    print("GBFS solution:")
    for i, node in enumerate(gbfs_solution):
        print(f"Step {i + 1}: {node}")


GBFS solution:
Step 1: ['w', 'c', 'g'][]Left
Step 2: ['w', 'c']['g']Right
Step 3: ['w', 'c']['g']Left
Step 4: ['c']['g', 'w']Right
Step 5: ['c', 'g']['w']Left
Step 6: ['g']['w', 'c']Right
Step 7: ['g']['w', 'c']Left
Step 8: []['w', 'c', 'g']Right


### A* Search

In [13]:
def astar(root_node):
    to_visit = [root_node]
    node = root_node
    previous_states = []
    parent_map = {root_node: None}
    while to_visit:
        node = heapq.heappop(to_visit)
        if not WGC_Node.state_in_previous(previous_states, node.west, node.east, node.boat_side):
            previous_states.append(node)
        node.generate_children(previous_states, parent_map)
        to_visit = node.children + to_visit
        if sorted(node.east) == ["c", "g", "w"]:
            solution = []
            while node is not None:
                solution = [node] + solution
                node = parent_map[node]
            return solution
    return None

if __name__ == "__main__":
    root = WGC_Node()
    astar_solution = astar(root)
    print("A* solution:")
    for i, node in enumerate(astar_solution):
        print(f"Step {i + 1}: {node}")


A* solution:
Step 1: ['w', 'c', 'g'][]Left
Step 2: ['w', 'c']['g']Right
Step 3: ['w', 'c']['g']Left
Step 4: ['c']['g', 'w']Right
Step 5: ['c', 'g']['w']Left
Step 6: ['g']['w', 'c']Right
Step 7: ['g']['w', 'c']Left
Step 8: []['w', 'c', 'g']Right
